In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("phenomsg/waste-classification")

print("Path to dataset files:", path)

In [2]:
!mv /root/.cache/kagglehub/datasets/phenomsg/waste-classification/versions/1 ./waste_dataset

In [3]:
import tensorflow as tf
import numpy as np
import os
import shutil
import zipfile
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from google.colab import files

# ======================================================
# 1. PATH TO YOUR DATASET
# ======================================================

# Make sure your folder is like:
# waste_dataset/
#   ├── Hazardous/
#   ├── Non-Recyclable/
#   ├── Organic/
#   ├── Recyclable/

DATASET_DIR = "/content/waste_dataset"

# ======================================================
# 2. IMAGE DATA GENERATORS
# ======================================================
datagen = ImageDataGenerator(
    rescale=1./255,
    zoom_range=0.2,
    rotation_range=20,
    horizontal_flip=True,
    validation_split=0.2
)

train_gen = datagen.flow_from_directory(
    DATASET_DIR,
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical",
    subset="training"
)

val_gen = datagen.flow_from_directory(
    DATASET_DIR,
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical",
    subset="validation"
)

class_names = list(train_gen.class_indices.keys())
print("Class Mapping:", train_gen.class_indices)

# Save class labels for later use
os.makedirs("model_package", exist_ok=True)
with open("model_package/labels.txt", "w") as f:
    for label in class_names:
        f.write(label + "\n")

# ======================================================
# 3. BUILD MODEL (MobileNetV2)
# ======================================================
base_model = MobileNetV2(include_top=False, input_shape=(224, 224, 3), weights="imagenet")
base_model.trainable = False

x = GlobalAveragePooling2D()(base_model.output)
x = Dropout(0.3)(x)
output = Dense(len(class_names), activation="softmax")(x)

model = Model(inputs=base_model.input, outputs=output)

model.compile(
    loss="categorical_crossentropy",
    optimizer=Adam(0.0005),
    metrics=["accuracy"]
)

model.summary()

# ======================================================
# 4. TRAIN
# ======================================================
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=12
)

# ======================================================
# 5. SAVE MODEL
# ======================================================
model.save("model_package/waste_model.h5")
print("Model saved!")

# ======================================================
# 6. ZIP THE MODEL PACKAGE FOR DOWNLOAD
# ======================================================
zip_filename = "waste_model_package.zip"

with zipfile.ZipFile(zip_filename, "w") as zipf:
    for foldername, subfolders, filenames in os.walk("model_package"):
        for filename in filenames:
            file_path = os.path.join(foldername, filename)
            zipf.write(file_path)

print("ZIP created:", zip_filename)

# ======================================================
# 7. DOWNLOAD ZIP
# ======================================================
files.download(zip_filename)

# ======================================================
# 8. FUNCTION FOR PREDICTING A CUSTOM IMAGE
# ======================================================
def predict_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_arr = image.img_to_array(img) / 255.0
    img_arr = np.expand_dims(img_arr, axis=0)

    pred = model.predict(img_arr)[0]
    idx = np.argmax(pred)

    print("Prediction:", class_names[idx])
    print("Probabilities:", pred)


Found 2309 images belonging to 4 classes.
Found 575 images belonging to 4 classes.
Class Mapping: {'Hazardous': 0, 'Non-Recyclable': 1, 'Organic': 2, 'Recyclable': 3}
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 112, 112,  │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 112, 112,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 112, 112,  │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 112, 112,  │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 112, 112,  │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 112, 112,  │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 113, 113,  │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 56, 56,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 56, 56,    │      2,304 │ block_1_depthwis

 Total params: 2,263,108 (8.63 MB)

 Trainable params: 5,124 (20.02 KB)

...

/usr/local/lib/python3.12/dist-packages/PIL/TiffImagePlugin.py:950: UserWarning: Truncated File Read
  warnings.warn(str(msg))


73/73 ━━━━━━━━━━━━━━━━━━━━ 123s 1s/step - accuracy: 0.3320 - loss: 1.6503 - val_accuracy: 0.4157 - val_loss: 1.3608
Epoch 2/12
73/73 ━━━━━━━━━━━━━━━━━━━━ 77s 1s/step - accuracy: 0.5678 - loss: 1.0995 - val_accuracy: 0.4539 - val_loss: 1.3725
Epoch 3/12
73/73 ━━━━━━━━━━━━━━━━━━━━ 77s 1s/step - accuracy: 0.6073 - loss: 0.9769 - val_accuracy: 0.4939 - val_loss: 1.2637
Epoch 4/12
73/73 ━━━━━━━━━━━━━━━━━━━━ 78s 1s/step - accuracy: 0.6639 - loss: 0.8869 - val_accuracy: 0.5148 - val_loss: 1.2631
Epoch 5/12
73/73 ━━━━━━━━━━━━━━━━━━━━ 77s 1s/step - accuracy: 0.6841 - loss: 0.8116 - val_accuracy: 0.5026 - val_loss: 1.3163
Epoch 6/12
73/73 ━━━━━━━━━━━━━━━━━━━━ 77s 1s/step - accuracy: 0.6961 - loss: 0.7935 - val_accuracy: 0.5183 - val_loss: 1.2905
Epoch 7/12
73/73 ━━━━━━━━━━━━━━━━━━━━ 79s 1s/step - accuracy: 0.7124 - loss: 0.7520 - val_accuracy: 0.5130 - val_loss: 1.3376
Epoch 8/12
73/73 ━━━━━━━━━━━━━━━━━━━━ 81s 1s/step - accuracy: 0.7193 - loss: 0.7422 - val_accuracy: 0.5078 - val_loss: 1.3137
Ep

Model saved!
ZIP created: waste_model_package.zip


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [3]:
import numpy as np
import tensorflow as tf
from google.colab.output import eval_js
from base64 import b64decode
from PIL import Image
import io

from tensorflow.keras.utils import load_img, img_to_array

# Decode JS base64 image
def js_to_image(js_reply):
    image_bytes = b64decode(js_reply.split(',')[1])
    return Image.open(io.BytesIO(image_bytes))

# Preprocess
def preprocess_image(img, img_size=(224, 224)):
    img = img.resize(img_size)
    img = img_to_array(img)
    img = img / 255.0
    return np.expand_dims(img, axis=0)


def predict_from_camera(model, class_names):
    js = """
    async function takePhoto() {
      const div = document.createElement('div');
      const video = document.createElement('video');
      const button = document.createElement('button');
      button.textContent = 'Capture';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});
      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();
      div.appendChild(button);

      await new Promise((resolve) =>
        button.onclick = () => resolve()
      );

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();

      return canvas.toDataURL('image/jpeg', 0.8);
    }
    takePhoto();
    """

    # Capture image
    js_reply = eval_js(js)
    img = js_to_image(js_reply)
    img_batch = preprocess_image(img)

    # Predictions
    preds = model.predict(img_batch)[0]

    # Print all probabilities
    print("\n--- Class Probabilities ---")
    for cls, prob in zip(class_names, preds):
        print(f"{cls}: {prob:.4f}")

    # Final prediction
    idx = np.argmax(preds)
    final_class = class_names[idx]
    confidence = preds[idx]

    print("\n--- Final Prediction ---")
    print(f"Predicted Class: {final_class}")
    print(f"Confidence: {confidence:.4f}")

    return preds, final_class, confidence


In [4]:
probs, predicted_class, confidence = predict_from_camera(model, class_names)

NameError: name 'model' is not defined